In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import copy
import os
import sys
import uuid
from importlib import reload
from pathlib import Path
sys.path.append(r'C:\dev\projects\weedget')

import ipyvuetify as v
import ipywidgets as w
import numpy as np
import pandas as pd
from ipywidgets import jslink
from IPython.display import display
from plotly import graph_objects as go
from plotly.subplots import make_subplots

from weedget import base as b, components as c

reload(c)
widgets = w
rng = np.random.default_rng(1988)

In [3]:
path = Path(r'M:\BHAI\Shared\MacroBot\Scrap_Results')

In [4]:
results_df = pd.read_csv(path / 'diff_smaller.csv', index_col=0).rename_axis('Feature')
pnl_df = pd.read_csv(path / 'diff_smaller_pl.csv', index_col=0, parse_dates=True, dayfirst=True).rename_axis('date').dropna()
features_df = pd.read_csv(path / 'diff_smaller_feat.csv', index_col=0, parse_dates=True, dayfirst=True).rename_axis('date').dropna()


In [5]:
n_cols = 10
n_rows = 1000
start = '1900-01-01'

df = pd.DataFrame(
    data=rng.normal(size=(n_rows, n_cols)),
    index=pd.DatetimeIndex(pd.date_range(start=start, periods=n_rows)),
    columns=[uuid.uuid4().hex[:10] for i in range(n_cols)]
)

In [21]:
dfc

DataFrameCombo(clicked_row={'Feature': 'US_Equity_Nasdaq_ZSc52', 'CumPL': 0.4843678655256274, 'Average_Long': …

In [6]:
dfc = c.DataFrameCombo(df=results_df, formats='{:.2%}')
out = w.Output()

In [17]:
def make_trace(series, name=None, **kwargs):
    scatter = go.Scatter(x=series.index, y=series, **kwargs)
    if name:
        scatter.name = name
    return scatter

def show_plot(event):
    print(event)
    data = event['new']
    out.clear_output()
    feature = data['Feature']    
    trace1 = make_trace(pnl_df[feature].cumsum(), 'strategy')
    trace2 = make_trace(pnl_df['Hold'].cumsum(), 'target')
    trace3 = make_trace(features_df[feature], 'feature', line_color='rgba(190, 190, 190,0.5)')
    fig = make_subplots(specs=[[{"secondary_y": True}]])
    fig.add_traces(data=[
        trace1,
        trace2,
    ])
    fig.add_trace(trace3, secondary_y=True)
    fig.update_layout(
        title=feature,
        height=700,
        legend=dict(
            orientation="h",
            yanchor="bottom",
            y=1.02,
            xanchor="right",
            x=1
        ),
    )
    
    
    fig.update_xaxes(
        rangeslider_visible=True,
        rangeselector=dict(
            buttons=list([
                dict(count=1, label="1m", step="month", stepmode="backward"),
                dict(count=6, label="6m", step="month", stepmode="backward"),
                dict(count=1, label="YTD", step="year", stepmode="todate"),
                dict(count=1, label="1y", step="year", stepmode="backward"),
                dict(step="all")
            ])
        )
    )
    
    with out:
        display(fig)
    

In [18]:
dfc.observe(show_plot, 'clicked_row')

In [19]:
sse_tab = v.Row(
    children=[
        v.Col(
            cols=6,
            children=[dfc]
        ),
        v.Col(
            cols=6,
            children=[out]
        )
    ]
)

In [20]:
sse_tab

Row(children=[Col(children=[DataFrameCombo(clicked_row={'Feature': 'EMU_Eonia_1m_ZSc52', 'CumPL': 0.7963250104…